In [ ]:
# ==============================================================================
# AUTHOR AND LICENSE INFORMATION
# ==============================================================================
# This code was developed by Dr. Tevfik Denizhan Müftüoğlu.
# It can be used and/or modified with the condition of providing proper citation.
#
# Bu kod Dr. Tevfik Denizhan Müftüoğlu tarafından geliştirilmiştir.
# Atıf verilmesi kaydıyla aynen ya da değiştirilerek kullanılabilir.
# ==============================================================================

# ==============================================================================
# STEP 1: INSTALL AND IMPORT LIBRARIES
# Adım 1: Kütüphaneleri Yükleme ve İçe Aktarma
# ==============================================================================
print("--- Installing required libraries... ---")
print("--- Gerekli kütüphaneler yükleniyor... ---")
!pip install pymannkendall -q
!pip install prophet -q
!pip install xgboost -q
!pip install lightgbm -q
!pip install graphviz -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymannkendall as mk
from scipy.stats import gumbel_r
from google.colab import files
import warnings
warnings.filterwarnings('ignore')
import os
import random

# Models and Metrics
from sklearn.ensemble import IsolationForest, RandomForestRegressor, StackingRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU
from graphviz import Digraph

# --- SET ALL RANDOM SEEDS FOR REPRODUCIBILITY ---
# --- TEKRARLANABİLİRLİK İÇİN TÜM RASTGELELİK TOHUMLARINI AYARLAMA ---
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
print("\n--- Random seeds set to 42 for reproducibility. ---")
print("--- Tekrarlanabilirlik için rastgelelik tohumları 42'ye ayarlandı. ---")

print("\n--- Libraries installed and imported. ---")
print("--- Kütüphaneler yüklendi ve içe aktarıldı. ---")

# ==============================================================================
# STEP 2: LOAD AND PREPARE THE DATA
# Adım 2: Veriyi Yükleme ve Hazırlama
# ==============================================================================
print("\n--- Please select the NASAPOWERDATASET.csv file for upload: ---")
print("--- Lütfen yüklemek için NASAPOWERDATASET.csv dosyasını seçin: ---")
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(file_name)
df['date'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['DOY'].astype(str), format='%Y-%j')
df.set_index('date', inplace=True)
print("\n--- Data successfully loaded and prepared. ---")
print("--- Veri başarıyla yüklendi ve hazırlandı. ---")
plt.style.use('seaborn-v0_8-whitegrid')

# ==============================================================================
# ANALYSIS PART A: INDEPENDENT STATISTICAL ANALYSES
# ANALİZ BÖLÜM A: BAĞIMSIZ İSTATİSTİKSEL ANALİZLER
# ==============================================================================
print("\n--- Starting Return Period Analysis (Gumbel)... ---")
print("--- Tekerrür Analizi (Gumbel) Başlatılıyor... ---")
annual_max_precip = df['PRECTOTCORR'].resample('Y').max()
loc, scale = gumbel_r.fit(annual_max_precip)
return_periods = np.array([2, 5, 10, 25, 50, 100])
design_precip = gumbel_r.ppf(1 - 1 / return_periods, loc=loc, scale=scale)
results_df_gumbel = pd.DataFrame({'Tekerrür Periyodu (Yıl)': return_periods, 'Tasarım Yağışı (mm/gün)': design_precip.round(2)})
results_df_gumbel.to_csv('design_rainfall_table.csv', index=False)
print("--- Return period analysis complete. Downloading 'design_rainfall_table.csv'... ---")
print("--- Tekerrür analizi tamamlandı. 'design_rainfall_table.csv' indiriliyor... ---")
files.download('design_rainfall_table.csv')

# --- YENİ VE DOĞRU TREND ANALİZİ ---
# --- NEW AND CORRECT TREND ANALYSIS ---
print("\n--- Starting Trend Analysis on Annual Maximum Precipitation... ---")
print("--- Yıllık Maksimum Yağış Üzerinde Trend Analizi Başlatılıyor... ---")
annual_max_precip_trend = df['PRECTOTCORR'].resample('Y').max()
mk_result_annual_max = mk.original_test(annual_max_precip_trend)
plt.figure(figsize=(12, 7))
plt.plot(annual_max_precip_trend.index.year, annual_max_precip_trend, marker='o', linestyle='-', label='Yıllık Maksimum Günlük Yağış')
trend_line = mk_result_annual_max.intercept + mk_result_annual_max.slope * annual_max_precip_trend.index.year
plt.plot(annual_max_precip_trend.index.year, trend_line, 'r--', label=f'Theil-Sen Trend (p-değeri: {mk_result_annual_max.p:.3f})')
plt.title('Yıllık Maksimum Günlük Yağış Miktarındaki Trend')
plt.xlabel('Yıl')
plt.ylabel('Maksimum Günlük Yağış (mm/gün)')
plt.legend()
plt.grid(True)
plt.savefig('annual_max_precip_trend.png', dpi=300, bbox_inches='tight')
print("--- Trend analysis complete. Downloading 'annual_max_precip_trend.png'... ---")
print("--- Trend analizi tamamlandı. 'annual_max_precip_trend.png' indiriliyor... ---")
files.download('annual_max_precip_trend.png')

# ==============================================================================
# ANALYSIS PART B: AUTOMATED FEATURE SELECTION & MODEL COMPETITION
# ANALİZ BÖLÜM B: OTOMATİK ÖZELLİK SEÇİMİ VE MODEL YARIŞMASI
# ==============================================================================
print("\n--- Starting Automated Feature Selection... ---")
print("--- Otomatik Özellik Seçimi Başlatılıyor... ---")
POTENTIAL_FEATURES = ['T2M', 'RH2M', 'PS', 'WS10M', 'WD10M', 'ALLSKY_SFC_SW_DWN']
TARGET_COL = 'PRECTOTCORR'
df_fs = df[[TARGET_COL] + POTENTIAL_FEATURES].copy().dropna()
for col in df_fs.columns: [df_fs.update({f'{col}_lag_{lag}': df_fs[col].shift(lag)}) for lag in range(1, 4)]
df_fs = df_fs.dropna()
X_fs = df_fs.drop(TARGET_COL, axis=1)
y_fs = df_fs[TARGET_COL]
explorer_model = xgb.XGBRegressor(n_estimators=100, n_jobs=-1, random_state=SEED).fit(X_fs, y_fs, verbose=False) # SEED added
importances = pd.DataFrame({'feature': X_fs.columns, 'importance': explorer_model.feature_importances_}).sort_values('importance', ascending=False)
TOP_FEATURES = importances['feature'].head(15).tolist()
plt.figure(figsize=(12, 10))
sns.barplot(x='importance', y='feature', data=importances.head(15)); plt.title('Feature Importance Ranking'); plt.xlabel('Importance'); plt.ylabel('Feature')
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
print(f"--- Top {len(TOP_FEATURES)} features selected. Downloading 'feature_importance.png'... ---")
print(f"--- En önemli {len(TOP_FEATURES)} özellik seçildi. 'feature_importance.png' indiriliyor... ---")
files.download('feature_importance.png')

print("\n--- Starting Final Model Competition (This may take a very long time)... ---")
print("--- Nihai Model Yarışması Başlatılıyor (Bu işlem ÇOK UZUN sürebilir)... ---")
# Data Prep
df_final = df[[TARGET_COL] + POTENTIAL_FEATURES].copy()
train_size = int(len(df_final) * 0.8)
train_df, test_df = df_final.iloc[:train_size], df_final.iloc[train_size:]
y_test_actual = test_df[TARGET_COL]
performance_results = []
X_train_tree, y_train_tree = X_fs.loc[train_df.index.min():train_df.index.max(), TOP_FEATURES], y_fs.loc[train_df.index.min():train_df.index.max()]
X_test_tree, y_test_tree = X_fs.loc[test_df.index.min():test_df.index.max(), TOP_FEATURES], y_fs.loc[test_df.index.min():test_df.index.max()]

# --- Model Training ---
print("\n1. Training All 11 Models...")
print("1. Tüm 11 Model Eğitiliyor...")
# Tree-based Models
model_xgb = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=50, eval_metric='rmse', random_state=SEED).fit(X_train_tree, y_train_tree, eval_set=[(X_test_tree, y_test_tree)], verbose=False) # SEED added
predict_xgb = model_xgb.predict(X_test_tree)
performance_results.append({'Model': 'XGBoost', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_tree, predict_xgb)), 'R-Kare (R²)': r2_score(y_test_tree, predict_xgb)})
model_lgbm = lgb.LGBMRegressor(n_estimators=1000, random_state=SEED); model_lgbm.fit(X_train_tree, y_train_tree, eval_set=[(X_test_tree, y_test_tree)], callbacks=[lgb.early_stopping(50, verbose=False)]) # SEED added
predict_lgbm = model_lgbm.predict(X_test_tree)
performance_results.append({'Model': 'LightGBM', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_tree, predict_lgbm)), 'R-Kare (R²)': r2_score(y_test_tree, predict_lgbm)})
model_rf = RandomForestRegressor(n_estimators=100, random_state=SEED, n_jobs=-1).fit(X_train_tree, y_train_tree) # SEED added
predict_rf = model_rf.predict(X_test_tree)
performance_results.append({'Model': 'Random Forest', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_tree, predict_rf)), 'R-Kare (R²)': r2_score(y_test_tree, predict_rf)})
# Prophet
FEATURES_FOR_PROPHET = [f for f in TOP_FEATURES if f in POTENTIAL_FEATURES and f in df_final.columns]
prophet_df = df_final.reset_index().rename(columns={'date': 'ds', TARGET_COL: 'y'})
train_prophet, test_prophet = prophet_df.iloc[:train_size], prophet_df.iloc[train_size:]
model_prophet = Prophet(); [model_prophet.add_regressor(col) for col in FEATURES_FOR_PROPHET]
model_prophet.fit(train_prophet)
predict_prophet = model_prophet.predict(test_prophet.drop('y', axis=1))['yhat'].values
performance_results.append({'Model': 'Prophet', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_actual, predict_prophet)), 'R-Kare (R²)': r2_score(y_test_actual, predict_prophet)})
# SARIMAX
X_sarimax_train = train_df[POTENTIAL_FEATURES]; X_sarimax_test = test_df[POTENTIAL_FEATURES]
model_sarimax = SARIMAX(train_df[TARGET_COL], exog=X_sarimax_train, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0)).fit(disp=False)
predict_sarimax = model_sarimax.get_forecast(steps=len(test_df), exog=X_sarimax_test).predicted_mean
performance_results.append({'Model': 'SARIMAX', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_actual, predict_sarimax)), 'R-Kare (R²)': r2_score(y_test_actual, predict_sarimax)})
# ANN
scaler_ann = StandardScaler(); X_train_ann = scaler_ann.fit_transform(X_train_tree); X_test_ann = scaler_ann.transform(X_test_tree)
model_ann = Sequential([Dense(64, activation='relu', input_shape=[X_train_ann.shape[1]]), Dense(32, activation='relu'), Dense(1)])
model_ann.compile(optimizer='adam', loss='mean_squared_error'); model_ann.fit(X_train_ann, y_train_tree, epochs=50, batch_size=32, verbose=0)
predict_ann = model_ann.predict(X_test_ann).flatten()
performance_results.append({'Model': 'ANN', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_tree, predict_ann)), 'R-Kare (R²)': r2_score(y_test_tree, predict_ann)})
# LSTM & GRU
FEATURES_FOR_DL = [TARGET_COL] + [f for f in TOP_FEATURES if f in POTENTIAL_FEATURES and f in df_final.columns]
scaler_dl = MinMaxScaler(); scaled_data = scaler_dl.fit_transform(df_final[FEATURES_FOR_DL])
train_scaled, test_scaled = scaled_data[:train_size], scaled_data[train_size:]
def create_multivariate_dataset(dataset, time_step=30):
    dataX, dataY = [], []; [dataX.append(dataset[i:(i + time_step), :]) or dataY.append(dataset[i + time_step, 0]) for i in range(len(dataset) - time_step - 1)]; return np.array(dataX), np.array(dataY)
time_step = 30
X_train_dl, y_train_dl = create_multivariate_dataset(train_scaled, time_step); X_test_dl, _ = create_multivariate_dataset(test_scaled, time_step)
model_lstm = Sequential([LSTM(50, input_shape=(X_train_dl.shape[1], X_train_dl.shape[2])), Dense(1)]); model_lstm.compile(optimizer='adam', loss='mean_squared_error'); model_lstm.fit(X_train_dl, y_train_dl, epochs=20, batch_size=64, verbose=0)
predict_lstm_scaled = model_lstm.predict(X_test_dl)
model_gru = Sequential([GRU(50, input_shape=(X_train_dl.shape[1], X_train_dl.shape[2])), Dense(1)]); model_gru.compile(optimizer='adam', loss='mean_squared_error'); model_gru.fit(X_train_dl, y_train_dl, epochs=20, batch_size=64, verbose=0)
predict_gru_scaled = model_gru.predict(X_test_dl)
dummy_lstm = np.zeros((len(predict_lstm_scaled), len(FEATURES_FOR_DL))); dummy_lstm[:,0] = predict_lstm_scaled.flatten(); predict_lstm = scaler_dl.inverse_transform(dummy_lstm)[:,0]
dummy_gru = np.zeros((len(predict_gru_scaled), len(FEATURES_FOR_DL))); dummy_gru[:,0] = predict_gru_scaled.flatten(); predict_gru = scaler_dl.inverse_transform(dummy_gru)[:,0]
y_test_dl_actual = y_test_actual.iloc[time_step+1:]
performance_results.append({'Model': 'LSTM', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_dl_actual, predict_lstm)), 'R-Kare (R²)': r2_score(y_test_dl_actual, predict_lstm)})
performance_results.append({'Model': 'GRU', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_dl_actual, predict_gru)), 'R-Kare (R²)': r2_score(y_test_dl_actual, predict_gru)})
# Hybrid Models (Base and Optimized)
df_hybrid = df_final.reset_index().rename(columns={'date':'ds', TARGET_COL:'y'})
train_hybrid_prophet = df_hybrid.iloc[:train_size]
model_prophet_base = Prophet().fit(train_hybrid_prophet[['ds', 'y']])
full_forecast = model_prophet_base.predict(df_hybrid[['ds']])
df_hybrid['prophet_forecast'] = full_forecast['yhat']
df_hybrid['residuals'] = df_hybrid['y'] - df_hybrid['prophet_forecast']
df_temp_for_lags = df_final.copy()
for col in [TARGET_COL] + POTENTIAL_FEATURES: [df_temp_for_lags.update({f'{col}_lag_{lag}': df_temp_for_lags[col].shift(lag)}) for lag in range(1, 4)]
df_temp_for_lags = df_temp_for_lags.dropna()
df_hybrid_xgb = df_temp_for_lags.reset_index().rename(columns={'date':'ds'})
df_hybrid_xgb = pd.merge(df_hybrid_xgb, df_hybrid[['ds', 'prophet_forecast', 'residuals']], on='ds', how='inner').set_index('ds')
FEATURES_HYBRID = [f for f in df_hybrid_xgb.columns if f not in ['y', 'residuals', 'prophet_forecast', 'ds', TARGET_COL]]
TARGET_HYBRID = 'residuals'
train_size_hybrid = int(len(df_hybrid_xgb) * 0.8)
X_train_h, X_test_h = df_hybrid_xgb.iloc[:train_size_hybrid][FEATURES_HYBRID], df_hybrid_xgb.iloc[train_size_hybrid:][FEATURES_HYBRID]
y_train_h, y_test_h = df_hybrid_xgb.iloc[:train_size_hybrid][TARGET_HYBRID], df_hybrid_xgb.iloc[train_size_hybrid:][TARGET_HYBRID]
prophet_test_forecast = df_hybrid_xgb['prophet_forecast'].iloc[train_size_hybrid:]
y_test_hybrid_actual = y_test_h + prophet_test_forecast
model_xgb_hybrid_base = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=50, eval_metric='rmse', random_state=SEED).fit(X_train_h, y_train_h, eval_set=[(X_test_h, y_test_h)], verbose=False) # SEED added
predict_residuals_base = model_xgb_hybrid_base.predict(X_test_h)
predict_hybrid_base = prophet_test_forecast + predict_residuals_base
performance_results.append({'Model': 'Hybrid (Base)', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_hybrid_actual, predict_hybrid_base)), 'R-Kare (R²)': r2_score(y_test_hybrid_actual, predict_hybrid_base)})
param_grid = {'n_estimators': [100, 500, 1000], 'max_depth': [3, 5, 10], 'learning_rate': [0.01, 0.05, 0.1], 'subsample': [0.7, 0.9, 1.0]}
random_search = RandomizedSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror', random_state=SEED), param_distributions=param_grid, n_iter=20, scoring='r2', cv=3, verbose=0, n_jobs=-1, random_state=SEED).fit(X_train_h, y_train_h) # SEED added
best_hybrid_xgb = random_search.best_estimator_
predict_residuals_optimized = best_hybrid_xgb.predict(X_test_h)
predict_hybrid_optimized = prophet_test_forecast + predict_residuals_optimized
performance_results.append({'Model': 'Hybrid (Optimized)', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_hybrid_actual, predict_hybrid_optimized)), 'R-Kare (R²)': r2_score(y_test_hybrid_actual, predict_hybrid_optimized)})
# Stacking Model
estimators = [('xgb', xgb.XGBRegressor(n_estimators=100, random_state=SEED)), ('lgbm', lgb.LGBMRegressor(n_estimators=100, random_state=SEED)), ('rf', RandomForestRegressor(n_estimators=50, random_state=SEED, n_jobs=-1))] # SEED added
stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacking_model.fit(X_train_tree, y_train_tree)
predict_stacking = stacking_model.predict(X_test_tree)
performance_results.append({'Model': 'Stacking Ensemble', 'RMSE (mm/gün)': np.sqrt(mean_squared_error(y_test_tree, predict_stacking)), 'R-Kare (R²)': r2_score(y_test_tree, predict_stacking)})

# ==============================================================================
# FINAL EVALUATION & VISUALIZATION
# NİHAİ DEĞERLENDİRME VE GÖRSELLEŞTİRME
# ==============================================================================
print("\n--- Calculating Final Performances and Creating All Graphics... ---")
print("--- Nihai Performanslar Hesaplanıyor ve Tüm Grafikler Oluşturuluyor... ---")
performance_df = pd.DataFrame(performance_results).round(3).sort_values('R-Kare (R²)', ascending=False)
print("\n--- Final Model Comparison Results: ---")
print("--- Nihai Model Karşılaştırma Sonuçları: ---")
print(performance_df)
performance_df.to_csv('ULTIMATE_FINAL_comparison.csv', index=False)
print("\n--- Downloading performance table 'ULTIMATE_FINAL_comparison.csv'... ---")
print("--- Performans tablosu 'ULTIMATE_FINAL_comparison.csv' indiriliyor... ---")
files.download('ULTIMATE_FINAL_comparison.csv')

# --- Visualization 1: Performance Bar Plot ---
plt.figure(figsize=(12, 8))
sns.barplot(x='R-Kare (R²)', y='Model', data=performance_df, palette='viridis')
plt.title('Final Model Performance Comparison (R-squared)', fontsize=16)
plt.xlabel('R-squared (R²)', fontsize=12); plt.ylabel('Model', fontsize=12)
plt.xlim(min(0, performance_df['R-Kare (R²)'].min()*1.1), max(performance_df['R-Kare (R²)']) * 1.1)
plt.savefig('model_performance_barchart.png', dpi=300, bbox_inches='tight')
print("\n--- Downloading performance bar chart 'model_performance_barchart.png'... ---")
print("--- Performans bar grafiği 'model_performance_barchart.png' indiriliyor... ---")
files.download('model_performance_barchart.png')

# --- Visualization 2: Time Series Comparison Plot ---
plt.figure(figsize=(18, 9))
plt.plot(y_test_tree.index, y_test_tree.values, label='Actual Values', color='k', linewidth=2, alpha=0.6)
plot_styles = {'XGBoost': {'color': 'purple', 'linestyle': '--'}, 'LightGBM': {'color': 'green', 'linestyle': ':'}, 'Random Forest': {'color': 'blue', 'linestyle': '-.'}, 'ANN': {'color': 'red', 'linestyle': '-'}, 'Stacking Ensemble': {'color': 'orange', 'linestyle': '--'}, 'Hybrid (Optimized)': {'color': 'cyan', 'linestyle': ':'}}
top_models_df = performance_df.head(6)
# Create a dictionary of predictions for easier lookup
all_predictions = {
    'XGBoost': pd.Series(predict_xgb, index=y_test_tree.index),
    'LightGBM': pd.Series(predict_lgbm, index=y_test_tree.index),
    'Random Forest': pd.Series(predict_rf, index=y_test_tree.index),
    'ANN': pd.Series(predict_ann, index=y_test_tree.index),
    'Stacking Ensemble': pd.Series(predict_stacking, index=y_test_tree.index),
    'Hybrid (Optimized)': pd.Series(predict_hybrid_optimized, index=y_test_hybrid_actual.index),
    'Hybrid (Base)': pd.Series(predict_hybrid_base, index=y_test_hybrid_actual.index),
    'Prophet': pd.Series(predict_prophet, index=y_test_actual.index),
    'SARIMAX': pd.Series(predict_sarimax, index=y_test_actual.index),
    'LSTM': pd.Series(predict_lstm, index=y_test_dl_actual.index),
    'GRU': pd.Series(predict_gru, index=y_test_dl_actual.index)
}
for index, row in top_models_df.iterrows():
    model_name = row['Model']
    if model_name in plot_styles:
        r2_val = row['R-Kare (R²)']
        prediction_series = all_predictions.get(model_name)
        if prediction_series is not None:
            plt.plot(prediction_series.index, prediction_series.values, label=f'{model_name} (R²={r2_val:.3f})', color=plot_styles[model_name]['color'], linestyle=plot_styles[model_name]['linestyle'])
plt.title('Final Comparison of Top Performing Models'); plt.xlabel('Date'); plt.ylabel('Precipitation (mm/day)'); plt.legend()
plt.savefig('ULTIMATE_FINAL_comparison.png', dpi=300, bbox_inches='tight')
print("\n--- Downloading final comparison plot 'ULTIMATE_FINAL_comparison.png'... ---")
print("--- Nihai karşılaştırma grafiği 'ULTIMATE_FINAL_comparison.png' indiriliyor... ---")
files.download('ULTIMATE_FINAL_comparison.png')


# --- Visualization 3: Actual vs. Predicted Scatter Plot for the Champion ---
champion_model_name = performance_df.iloc[0]['Model']
champion_predictions = all_predictions.get(champion_model_name)
if champion_predictions is not None:
    y_test_champion = y_test_hybrid_actual if 'Hybrid' in champion_model_name else (y_test_dl_actual if champion_model_name in ['LSTM', 'GRU'] else y_test_tree)
    # Align indices for correct plotting
    common_index = y_test_champion.index.intersection(champion_predictions.index)
    y_test_champion = y_test_champion[common_index]
    champion_predictions = champion_predictions[common_index]

    champion_r2 = performance_df.iloc[0]['R-Kare (R²)']
    plt.figure(figsize=(10, 10))
    plt.scatter(y_test_champion, champion_predictions, alpha=0.5, label='Model Predictions')
    min_val = 0; max_val = max(y_test_champion.max(), champion_predictions.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction (R²=1)')
    plt.title(f'Champion Model Performance: {champion_model_name} (R² = {champion_r2:.3f})', fontsize=16)
    plt.xlabel('Actual Precipitation (mm/day)'); plt.ylabel('Predicted Precipitation (mm/day)'); plt.legend(); plt.grid(True); plt.axis('equal'); plt.xlim(left=min_val, right=max_val); plt.ylim(bottom=min_val, top=max_val)
    plt.savefig('champion_model_scatter.png', dpi=300, bbox_inches='tight')
    print("\n--- Downloading champion model scatter plot 'champion_model_scatter.png'... ---")
    print("--- Şampiyon modelin dağılım grafiği 'champion_model_scatter.png' indiriliyor... ---")
    files.download('champion_model_scatter.png')


# --- Visualization 4: All Architectural Diagrams ---
print("\n--- Creating All Architectural Diagrams... ---")
print("--- Tüm Mimari Şemaları Oluşturuluyor... ---")
graph_attr = {'rankdir': 'TB', 'splines': 'ortho', 'nodesep': '0.8'}; node_attr = {'style': 'filled', 'shape': 'box', 'align': 'center', 'fontsize': '10', 'fontname': 'Helvetica'}; edge_attr = {'fontsize': '9', 'fontname': 'Helvetica'}
# Decision Tree & RF
rf_dot = Digraph('RandomForestConcept', graph_attr=graph_attr, node_attr=node_attr, edge_attr=edge_attr); rf_dot.attr(label='Random Forest Concept')
with rf_dot.subgraph(name='cluster_0') as c: c.attr(color='white'); c.node('Input', 'Full Training Data', shape='parallelogram', fillcolor='lightgrey')
with rf_dot.subgraph(name='cluster_1') as c: c.attr(label='Bootstrap Sampling & Feature Subspacing', style='dashed'); c.node('Data1', 'Data Subset 1\n(Random Features)'); c.node('Data2', 'Data Subset 2\n(Random Features)'); c.node('DataN', 'Data Subset N\n(Random Features)')
with rf_dot.subgraph(name='cluster_2') as c: c.attr(label='Independent Decision Trees', style='dashed'); c.node('Tree1', 'Decision Tree 1', fillcolor='lightblue'); c.node('Tree2', 'Decision Tree 2', fillcolor='lightblue'); c.node('TreeN', 'Decision Tree N', fillcolor='lightblue')
rf_dot.node('Avg', 'Averaging\nPredictions', shape='ellipse', fillcolor='orange'); rf_dot.node('Output', 'Final Prediction', shape='parallelogram', fillcolor='gold')
rf_dot.edge('Input', 'Data1'); rf_dot.edge('Input', 'Data2'); rf_dot.edge('Input', 'DataN'); rf_dot.edge('Data1', 'Tree1'); rf_dot.edge('Data2', 'Tree2'); rf_dot.edge('DataN', 'TreeN'); rf_dot.edge('Tree1', 'Avg'); rf_dot.edge('Tree2', 'Avg'); rf_dot.edge('TreeN', 'Avg'); rf_dot.edge('Avg', 'Output')
rf_dot.render('random_forest_concept', format='png', view=False); files.download('random_forest_concept.png')
# ANN (MLP)
ann_dot = Digraph('ANN_Architecture', graph_attr={'rankdir': 'LR', 'splines': 'line'}, node_attr=node_attr, edge_attr=edge_attr); ann_dot.attr(label='Artificial Neural Network (ANN) Architecture')
with ann_dot.subgraph(name='cluster_0') as c: c.attr(label='Input Layer'); c.attr(color='white'); c.node_attr.update(shape='parallelogram', fillcolor='lightgrey'); c.node('xn', f'Input Features\n({len(TOP_FEATURES)} Features)')
with ann_dot.subgraph(name='cluster_1') as c: c.attr(label='Hidden Layer 1'); c.attr(color='white'); c.node_attr.update(shape='circle', fillcolor='lightblue'); c.node('h1_1', 'Neuron'); c.node('h1_n', '...')
with ann_dot.subgraph(name='cluster_2') as c: c.attr(label='Hidden Layer 2'); c.attr(color='white'); c.node_attr.update(shape='circle', fillcolor='lightblue'); c.node('h2_1', 'Neuron'); c.node('h2_n', '...')
with ann_dot.subgraph(name='cluster_3') as c: c.attr(label='Output Layer'); c.attr(color='white'); c.node_attr.update(shape='parallelogram', fillcolor='gold'); c.node('y', 'Precipitation\nForecast')
ann_dot.edge('xn', 'h1_1'); ann_dot.edge('xn', 'h1_n'); ann_dot.edge('h1_1', 'h2_1'); ann_dot.edge('h1_n', 'h2_n'); ann_dot.edge('h2_1', 'y'); ann_dot.edge('h2_n', 'y')
ann_dot.render('ann_architecture', format='png', view=False); files.download('ann_architecture.png')
# RNN Concept
rnn_dot = Digraph('RNN_Concept', graph_attr=graph_attr, node_attr=node_attr, edge_attr=edge_attr); rnn_dot.attr(label='Recurrent Neural Network (RNN) Concept for LSTM/GRU')
rnn_dot.node('xt', 'Input at time (t)\n(Today\'s Data)', shape='parallelogram', fillcolor='lightgrey'); rnn_dot.node('ht', 'Output at time (t)\n(Today\'s Prediction)', shape='parallelogram', fillcolor='gold')
rnn_dot.node('A', 'Neural Network\n(Hidden State)', fillcolor='lightblue'); rnn_dot.edge('xt', 'A'); rnn_dot.edge('A', 'ht'); rnn_dot.edge('A', 'A', label='Memory from (t-1)\n(Yesterday\'s Information)')
rnn_dot.render('recurrent_neural_network_concept', format='png', view=False); files.download('recurrent_neural_network_concept.png')
# Stacking Ensemble
stack_dot = Digraph('Stacking_Architecture', graph_attr=graph_attr, node_attr=node_attr, edge_attr=edge_attr); stack_dot.attr(label='Stacking Ensemble Architecture')
stack_dot.node('Input', f'Input Features\n({len(TOP_FEATURES)} Features)', shape='parallelogram', fillcolor='lightgrey')
with stack_dot.subgraph(name='cluster_base') as c: c.attr(label='Layer 0: Diverse Base Models', style='dashed'); c.node('XGB', 'XGBoost'); c.node('RF', 'Random Forest'); c.node('LGBM', 'LightGBM')
stack_dot.node('Preds', 'Base Model Predictions\n(New Features)', shape='parallelogram', fillcolor='orange')
with stack_dot.subgraph(name='cluster_meta') as c: c.attr(label='Layer 1: Meta-Model', style='dashed'); c.node('Meta', 'Linear Regression')
stack_dot.node('Output', 'Final Prediction', shape='parallelogram', fillcolor='gold')
stack_dot.edge('Input', 'XGB'); stack_dot.edge('Input', 'RF'); stack_dot.edge('Input', 'LGBM'); stack_dot.edge('XGB', 'Preds'); stack_dot.edge('RF', 'Preds'); stack_dot.edge('LGBM', 'Preds'); stack_dot.edge('Preds', 'Meta'); stack_dot.edge('Meta', 'Output')
stack_dot.render('stacking_architecture', format='png', view=False)
print("--- Downloading all architectural diagrams... ---")
print("--- Tüm mimari şemaları indiriliyor... ---")
files.download('stacking_architecture.png')

print("\n\n=== ALL ANALYSES AND VISUALIZATIONS COMPLETED ===\n=== TÜM ANALİZLER VE GÖRSELLEŞTİRMELER TAMAMLANDI ===")